In [ ]:
#  Flow of model
# 1. Data Preprocessing 
# 2. Model.
# 3. Loss/optimization
# 4. Training. 

In [ ]:
pip install pytorch-lightning

     |████████████████████████████████| 913 kB 12.0 MB/s 
     |████████████████████████████████| 829 kB 45.9 MB/s 
     |████████████████████████████████| 234 kB 57.4 MB/s 
     |████████████████████████████████| 10.6 MB 50.4 MB/s 
     |████████████████████████████████| 118 kB 53.0 MB/s 
     |████████████████████████████████| 636 kB 50.5 MB/s 
     |████████████████████████████████| 1.3 MB 42.3 MB/s 
     |████████████████████████████████| 142 kB 58.0 MB/s 
     |████████████████████████████████| 294 kB 58.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=6a885fdcfe18cc00c72cac35da40f10de9ef2c6b0f79c2a6d36ecfaca409923e
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninsta

In [ ]:
import numpy as np 
import torch 
import matplotlib.pyplot as plt
import sklearn
import torch.autograd
from torch.utils.data import DataLoader, random_split
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import SGD
from torch.optim import Adam
import os
from torchvision import datasets, transforms

if torch.cuda.is_available ():
  device = torch.device("cuda")
  print('GPU IS AVAILABLE :D') # Turn on GPU in collab 

else:
  
  device = torch.device("cpu")
  print('GPU not available ')



GPU not available 


Increased/ Customized data set - Training and Validation data 

In [ ]:
# Data preprocessing 
np.random.seed(50)
Inputs = 3*np.random.random(size = (300,4)) # can generate m = 40 samples and n = 4 features 
print(Inputs)
print(Inputs.shape)
#nputs_vec = Inputs.reshape(-1,1) # (1200 , 1) column vector 

[[1.48380494 0.68424931 0.76642177 1.18898973]
 [1.13194529 2.98972269 1.2245916  2.31568198]
 [2.28161006 0.93002805 1.03962361 1.05529445]
 ...
 [2.59049077 0.68359399 2.82602449 1.5580974 ]
 [0.56272659 0.21924999 0.22727439 0.6997265 ]
 [0.97621273 1.96945661 1.68518757 2.96951736]]
(300, 4)


In [ ]:
x_torch = torch.from_numpy(Inputs) # tensor
x_torch.requires_grad_(True) # graident for backprop 
x_torch.retain_grad() # leaf tensor is a root of the tree 
print(x_torch.shape)

torch.Size([300, 4])


In [ ]:
# returns labels
def l_labels(x):
  return x[0]**4 + (x[1])**3 + torch.sin(x[2]) + x[3]**2

In [ ]:
y = torch.zeros(size = (300 , 1)) # samples

for i in range(x_torch.shape[0]):
  y[i] = l_labels(x_torch[i])

In [ ]:
f_o = torch.zeros(size = (300,4))

for i in range(x_torch.shape[0]):
  f_o[i] = torch.autograd.functional.jacobian(l_labels , x_torch[i])

print(f_o.shape)
  
print(f_o)

torch.Size([300, 4])
tensor([[13.0674,  1.4046,  0.7204,  2.3780],
        [ 5.8014, 26.8153,  0.3393,  4.6314],
        [47.5099,  2.5949,  0.5065,  2.1106],
        ...,
        [69.5354,  1.4019, -0.9506,  3.1162],
        [ 0.7128,  0.1442,  0.9743,  1.3995],
        [ 3.7213, 11.6363, -0.1141,  5.9390]])


In [ ]:
print(x_torch.shape) # m =300, n =4 so 300 x 4
print(y.shape) # m = 300
print(f_o.shape) # m =300,, n = 4 so 300 x 4


torch.Size([300, 4])
torch.Size([300, 1])
torch.Size([300, 4])


In [ ]:
BATCH_SIZE = 25

In [ ]:
# Data Collected and pre-processing 

x = x_torch.type(torch.FloatTensor)
y = y.type(torch.FloatTensor)
f_o = f_o.type(torch.FloatTensor)


# Organise inputs, labels and differential labels in training and validation data set 

# Training to validation split is 5:1 ratio 
# Training data 
x_train = x[:250].to(device)
y_train = y[:250].to(device)
f_o_train = f_o[:250].to(device)



# Validation Data 
x_valid = x[250:].to(device)
y_valid = y[250:].to(device)
f_o_valid = f_o[250:].to(device)



print(x_train.shape)
print(y_train.shape)
print(f_o_train.shape)

print(x_valid.shape)
print(y_valid.shape)
print(f_o_valid.shape)




# trainset_set = torch.utils.data.TensorDataset(x_train, y_train, dy_dx_train)
# validation_set = torch.utils.data.TensorDataset(x_valid, y_valid, dy_dx_valid)

train_dataset = torch.utils.data.TensorDataset(x_train , y_train, f_o_train)
validation_dataset = torch.utils.data.TensorDataset(x_valid, y_valid, f_o_valid)


train_dataloader = torch.utils.data.DataLoader(train_dataset , batch_size=BATCH_SIZE , shuffle=False , num_workers =0)
val_dataloader = torch.utils.data.DataLoader(validation_dataset , batch_size = BATCH_SIZE, shuffle = False, num_workers = 0 )


torch.Size([250, 4])
torch.Size([250, 1])
torch.Size([250, 4])
torch.Size([50, 4])
torch.Size([50, 1])
torch.Size([50, 4])


In [ ]:
input_size = 4 # x falttended
hidden_size_1 = 36
hidden_size_2 = 36
output_size = 1 # y output of training 

learning_rate = 0.001 # hyperparameter
num_epochs = 50 # hyperparameter 


alpha = 1.5
beta = 1
delta = 0.5

In [ ]:
# class NeuralNet(nn.Module):
#   def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size): 
#     super(NeuralNet, self).__init__()
#     self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
#     self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
#     self.layer_3 = nn.Linear(hidden_size_2, output_size)

# # 3 layer MLP forward pass
#   def forward(self, x):
#     # layer 1
#     x = self.layer_1(x)
#     x = torch.Softplus(x)

#     # layer 2
#     x = self.layer_2(x)
#     x = torch.Softplus(x)

#     # layer 3 
#     x = self.layer_3(x)

#     return  x 




First Order Risk Model

In [ ]:
class TwinFirstOrderNetwork(pl.LightningModule):
  
  def __init__(self):
    super(TwinFirstOrderNetwork,self).__init__()
    self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
    self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
    self.layer_3 = nn.Linear(hidden_size_2, output_size)

    self.softplus = nn.Softplus() # activation functions
    self.loss = nn.MSELoss() # MSEloss()

    self.automatic_optimization = False # Turn off automatic optimization. Turns off backprop automatically on lighting so we can do backprop operations manually such as in pytorch

  def forward(self, x):
    # layer 1
    x = self.layer_1(x)
    x = self.softplus(x)

    # layer 2
    x = self.layer_2(x)
    x = self.softplus(x)

    # layer 3 
    x = self.layer_3(x)

    return x


  def training_step(self, train_batch, batch_idx):
    opt = self.optimizers()
    opt.zero_grad()

    x, y, f_o  = train_batch
    predicted_y = self.forward(x)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4))
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True)

    loss = alpha * self.loss(y, predicted_y) + beta * self.loss( f_o, predicted_f_o) 

    loss.backward(retain_graph=True) # keeps gradeints the same and not zero when first backprop is done so we can backprop over again (two backprops to train to get second order derivatives) 

    opt.step() # Every batch. 

    self.log('train_loss', loss)
    print(loss)
    return loss
    


  def validation_step(self, val_batch, batch_idx):
    x, y, f_o = val_batch
    predicted_y = self.forward(x)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4))
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True) # https://github.com/pytorch/pytorch/issues/46918 - Can show we need create_graph to be true as it backpropagets through the graph

    loss = alpha * self.loss(y, predicted_y) + beta * self.loss( f_o, predicted_f_o) 
    self.log('val_loss', loss)

    return loss


  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-3)
   



In [ ]:
trainer = pl.Trainer(max_epochs= 50)
# https://www.machinecurve.com/index.php/question/how-to-set-number-of-epochs-in-pytorch-lightning/
model = TwinFirstOrderNetwork() # Trained. 

trainer.fit(model, train_dataloader, val_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name     | Type     | Params
--------------------------------------
0 | layer_1  | Linear   | 180   
1 | layer_2  | Linear   | 1.3 K 
2 | layer_3  | Linear   | 37    
3 | softplus | Softplus | 0     
4 | loss     | MSELoss  | 0     
--------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


tensor(2673.7861, grad_fn=<AddBackward0>)
tensor(1793.2131, grad_fn=<AddBackward0>)
tensor(1248.1915, grad_fn=<AddBackward0>)
tensor(1691.8483, grad_fn=<AddBackward0>)
tensor(1707.0090, grad_fn=<AddBackward0>)
tensor(3210.1606, grad_fn=<AddBackward0>)
tensor(1844.8339, grad_fn=<AddBackward0>)
tensor(877.8954, grad_fn=<AddBackward0>)
tensor(2674.5598, grad_fn=<AddBackward0>)
tensor(2271.3281, grad_fn=<AddBackward0>)


tensor(2607.6870, grad_fn=<AddBackward0>)
tensor(1745.2805, grad_fn=<AddBackward0>)
tensor(1206.2043, grad_fn=<AddBackward0>)
tensor(1642.4736, grad_fn=<AddBackward0>)
tensor(1662.4850, grad_fn=<AddBackward0>)
tensor(3139.9612, grad_fn=<AddBackward0>)
tensor(1790.8125, grad_fn=<AddBackward0>)
tensor(842.7370, grad_fn=<AddBackward0>)
tensor(2612.3408, grad_fn=<AddBackward0>)
tensor(2210.8105, grad_fn=<AddBackward0>)


tensor(2541.3540, grad_fn=<AddBackward0>)
tensor(1697.2300, grad_fn=<AddBackward0>)
tensor(1163.9009, grad_fn=<AddBackward0>)
tensor(1592.1387, grad_fn=<AddBackward0>)
tensor(1617.0314, grad_fn=<AddBackward0>)
tensor(3066.4785, grad_fn=<AddBackward0>)
tensor(1734.0896, grad_fn=<AddBackward0>)
tensor(806.1387, grad_fn=<AddBackward0>)
tensor(2545.5066, grad_fn=<AddBackward0>)
tensor(2145.3027, grad_fn=<AddBackward0>)


...

tensor(551.9960, grad_fn=<AddBackward0>)
tensor(423.5281, grad_fn=<AddBackward0>)
tensor(306.4836, grad_fn=<AddBackward0>)
tensor(325.8870, grad_fn=<AddBackward0>)
tensor(416.3370, grad_fn=<AddBackward0>)
tensor(678.1957, grad_fn=<AddBackward0>)
tensor(345.7614, grad_fn=<AddBackward0>)
tensor(246.0585, grad_fn=<AddBackward0>)
tensor(641.0317, grad_fn=<AddBackward0>)
tensor(451.8565, grad_fn=<AddBackward0>)


tensor(533.3014, grad_fn=<AddBackward0>)
tensor(412.0458, grad_fn=<AddBackward0>)
tensor(300.2061, grad_fn=<AddBackward0>)
tensor(314.8349, grad_fn=<AddBackward0>)
tensor(403.0007, grad_fn=<AddBackward0>)
tensor(652.0695, grad_fn=<AddBackward0>)
tensor(336.1681, grad_fn=<AddBackward0>)
tensor(243.2891, grad_fn=<AddBackward0>)
tensor(622.6285, grad_fn=<AddBackward0>)
tensor(440.2245, grad_fn=<AddBackward0>)


tensor(516.9503, grad_fn=<AddBackward0>)
tensor(402.4907, grad_fn=<AddBackward0>)
tensor(294.9500, grad_fn=<AddBackward0>)
tensor(305.9948, grad_fn=<AddBackward0>)
tensor(392.1198, grad_fn=<AddBackward0>)
tensor(629.1791, grad_fn=<AddBackward0>)
tensor(328.6246, grad_fn=<AddBackward0>)
tensor(241.2319, grad_fn=<AddBackward0>)
tensor(606.9270, grad_fn=<AddBackward0>)
tensor(430.1494, grad_fn=<AddBackward0>)


tensor(502.8514, grad_fn=<AddBackward0>)
tensor(394.5141, grad_fn=<AddBackward0>)
tensor(290.3936, grad_fn=<AddBackward0>)
tensor(299.0330, grad_fn=<AddBackward0>)
tensor(383.3209, grad_fn=<AddBackward0>)
tensor(609.4499, grad_fn=<AddBackward0>)
tensor(322.7965, grad_fn=<AddBackward0>)
tensor(239.4728, grad_fn=<AddBackward0>)
tensor(593.6995, grad_fn=<AddBackward0>)
tensor(421.3247, grad_fn=<AddBackward0>)


tensor(490.7370, grad_fn=<AddBackward0>)
tensor(387.7093, grad_fn=<AddBackward0>)
tensor(286.2505, grad_fn=<AddBackward0>)
tensor(293.5514, grad_fn=<AddBackward0>)
tensor(376.1462, grad_fn=<AddBackward0>)
tensor(592.5895, grad_fn=<AddBackward0>)
tensor(318.3059, grad_fn=<AddBackward0>)
tensor(237.6982, grad_fn=<AddBackward0>)
tensor(582.5793, grad_fn=<AddBackward0>)
tensor(413.4391, grad_fn=<AddBackward0>)


In [ ]:
# # Neural Network 
# # Pytroch lighting 
# class model(pl.LightningModule):
#   def __init__(self):
#     super(model, self).__init__() 
#     self.fc1 = nn.linear(input_size, hidden_size_1)
#     self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
#     self.out = nn.Linear(hidden_size_2, output_size)
#     self.lr = 0.001
#     self.loss_1 = nn.MSELoss() # lOSS OF THE LABELS ALONE 
#     self.loss_2 = nn.MSEloss() # LOSS OF FIRST ORDER DIFFERENTIAL LABELS AND LABELS TOGETHER. 

# # 3 layer NEURAL NETWORK
# def forward(self, x):
#   batch_size, _ = x.size() # this is subject to change depends on input data 
#   x = x.view(batch_size, -1)
#   x = F.softplus(self.fc1)
#   x = F.softplus(self.fc2)
#   return self.out(x) 

# def configure_optimizers(self):
#   return Adam(self.paramters(), lr = self.lr) # return adam 

# def training_step(self, train_batch): 
#   x , y = train_batch  
#   prediction_prices = self.forward(x)
#   loss = alpha* self.loss_1(prediction_prices, y) + beta * self.loss_2


# class Data(plt.LightningModule):
#   def prepare_data(self):
#     self.









# # Back prop / losss 


# # Training 
# #validation 